# Cleaning and Filtering, Analysis of results

In [134]:
import json
import pandas as pd

In [136]:
with open('data/data.json', 'r') as f:
    data = json.load(f)

In [137]:
len(data)

47800

## 1. Looking for fraudulent takedows

Fraudulent takedown requests usually follow a similar pattern. They use as a `copyrighted_url` the page of an anonymous blog that was created for that purpose. Shady reputation management companies copy-paste contents that they want to be removed, backdate them and then alleged copyright infringement to Google.

For that reason, a good starting point to try identify fraudulent takedowns is to isolate those requests that have a blogging platform as a `copyrighted_url`.

## 1.1. Keeping only notices that originated from blogs

In [138]:
blog_words = [
    'blogspot',
    'tumblr',
    'livejournal',
    'issuu',
    'wix',
    'over-blog'
]

In [139]:
blogs_data = []


for item in data:
    
    if 'dmca' in item:
    
        if 'works' in item['dmca']:

            lumen_id = item['dmca']['id']
            copyrighted_urls = item['dmca']['works'][0]['copyrighted_urls']

            for copyrighted_url in copyrighted_urls:
                url = copyrighted_url['url']

                if any (blog_word in url for blog_word in blog_words):    
                    blogs_data.append(item)
                    break


# data[0]['dmca']['works'][0]['copyrighted_urls'][0]['url']

In [140]:
len(blogs_data)

582

There are 582 requests that were accepted by Google in 2023 and appear to have originated from blogging platforms. We need to take a closer look at them. To do so, let's flatten the json.

## 1.2 Flattening, Cleaning

In [49]:
!pip install flatten_json

  Preparing metadata (setup.py) ... done
  DEPRECATION: flatten_json is being installed using the legacy 'setup.py install' method, because it does not have a 'pyproject.toml' and the 'wheel' package is not installed. pip 23.1 will enforce this behaviour change. A possible replacement is to enable the '--use-pep517' option. Discussion can be found at https://github.com/pypa/pip/issues/8559
  Running setup.py install for flatten_json ... done

[notice] A new release of pip is available: 23.0.1 -> 23.2.1
[notice] To update, run: pip install --upgrade pip


In [50]:
from flatten_json import flatten

In [67]:
data_flattened = [flatten(blog_data) for blog_data in blogs_data]
df = pd.DataFrame(data_flattened)

In [68]:
df.head()

dmca_id dmca_type                            dmca_title dmca_body  \
0  30194140      DMCA  DMCA (Copyright) Complaint to Google      None   
1  30150668      DMCA  DMCA (Copyright) Complaint to Google      None   
2  30150676      DMCA  DMCA (Copyright) Complaint to Google      None   
3  30176050      DMCA  DMCA (Copyright) Complaint to Google      None   
4  32934442      DMCA  DMCA (Copyright) Complaint to Google      None   

             dmca_date_sent        dmca_date_received dmca_topics_0  \
0  2023-01-04T00:00:00.000Z  2023-01-04T00:00:00.000Z  DMCA Notices   
1  2023-01-01T00:00:00.000Z  2023-01-01T00:00:00.000Z  DMCA Notices   
2  2023-01-01T00:00:00.000Z  2023-01-01T00:00:00.000Z  DMCA Notices   
3  2023-01-03T00:00:00.000Z  2023-01-03T00:00:00.000Z  DMCA Notices   
4  2023-01-05T00:00:00.000Z  2023-01-05T00:00:00.000Z  DMCA Notices   

  dmca_topics_1     dmca_sender_name dmca_principal_name dmca_recipient_name  \
0     Copyright       Pengetahuan-ID                None          Google LLC   
1     Copyright              Vũ Đoàn                None          Google LLC   
2     Copyright              Vũ Đoàn                None          Google LLC   
3     Copyright  Raining Romance LLC                None          Google LLC   
4     Copyright           [REDACTED]                None          Google LLC   

                            dmca_works_0_description  \
0  Artikel blog saya dicuri yang berjudul Kumpula...   
1  The thief changed the URLs of the previously r...   
2  The thief changed the URLs of the previously r...   
3  Its my book. A black background with gold crac...   
4                                         [REDACTED]   

                  dmca_works_0_infringing_urls_0_url  \
0  https://www.hukum96.com/2021/08/wallpaper-cosp...   
1   https://yeumaivang.com/benh-ri-sat-tren-cay-mai/   
2  https://yeumaivang.com/cach-bon-voi-cho-cay-ma...   
3                  https://vk.com/wall329000261_1450   
4  https://es.cointelegraph.com/news/top-criptono...   

                 dmca_works_0_copyrighted_urls_0_url dmca_tags  \
0  https://pengetahuandi.blogspot.com/2019/12/kum...        []   
1  https://hoamaixunau.blogspot.com/2021/05/trieu...        []   
2  https://hoamaixunau.blogspot.com/2021/12/hieu-...        []   
3  https://www.epub.pub/book/eternally-hers-by-ja...        []   
4  https://www.tumblr.com/24newspressblog/7055967...        []   

  dmca_jurisdictions_0 dmca_action_taken dmca_language dmca_case_id_number  \
0                   ID              None          None                None   
1                   VN              None          None                None   
2                   VN              None          None                None   
3                   US              None          None                None   
4                   ES              None          None                None   

                            dmca_works_1_description  \
0                                                NaN   
1  The thief changed the URLs of the previously r...   
2  The thief changed the URLs of the previously r...   
3                                                NaN   
4                                                NaN   

                  dmca_works_1_infringing_urls_0_url  \
0                                                NaN   
1  https://yeumaivang.com/cac-benh-tren-cay-mai-v...   
2  https://yeumaivang.com/thoi-diem-de-bung-cay-m...   
3                                                NaN   
4                                                NaN   

                 dmca_works_1_copyrighted_urls_0_url  \
0                                                NaN   
1  https://hoamaixunau.blogspot.com/2020/05/dung-...   
2  https://hoamaixunau.blogspot.com/2020/06/thoi-...   
3                                                NaN   
4                                                NaN   

                            dmca_works_2_description  \
0                                                N

There's a bit of mess there. Let's clean our flattened df.

In [141]:
# Identifying columns related to infringing_urls and copyrighted_urls
infringing_url_columns = [col for col in df.columns if "infringing_urls" in col]
copyrighted_url_columns = [col for col in df.columns if "copyrighted_urls" in col]

# Melting the infringing_url_columns
infringing_urls_melted = df.melt(id_vars=['dmca_id'], value_vars=infringing_url_columns, value_name='infringing_url')
infringing_urls_melted = infringing_urls_melted.dropna(subset=['infringing_url']).drop(columns=['variable'])

# Melting the copyrighted_url_columns
copyrighted_urls_melted = df.melt(id_vars=['dmca_id'], value_vars=copyrighted_url_columns, value_name='copyrighted_url')
copyrighted_urls_melted = copyrighted_urls_melted.dropna(subset=['copyrighted_url']).drop(columns=['variable'])

infringing_urls_melted.head(), copyrighted_urls_melted.head()

(    dmca_id                                     infringing_url
 0  30194140  https://www.hukum96.com/2021/08/wallpaper-cosp...
 1  30150668   https://yeumaivang.com/benh-ri-sat-tren-cay-mai/
 2  30150676  https://yeumaivang.com/cach-bon-voi-cho-cay-ma...
 3  30176050                  https://vk.com/wall329000261_1450
 4  32934442  https://es.cointelegraph.com/news/top-criptono...,
     dmca_id                                    copyrighted_url
 0  30194140  https://pengetahuandi.blogspot.com/2019/12/kum...
 1  30150668  https://hoamaixunau.blogspot.com/2021/05/trieu...
 2  30150676  https://hoamaixunau.blogspot.com/2021/12/hieu-...
 3  30176050  https://www.epub.pub/book/eternally-hers-by-ja...
 4  32934442  https://www.tumblr.com/24newspressblog/7055967...)

In [142]:
# Keeping only the columns that are not related to infringing or copyrighted URLs
main_data = df.drop(columns=infringing_url_columns + copyrighted_url_columns)

# Merging the main data with the melted infringing and copyrighted URLs
tidy_data = main_data.merge(infringing_urls_melted, on='dmca_id', how='left').merge(copyrighted_urls_melted, on='dmca_id', how='left')

# Displaying the first few rows of the tidy data
tidy_data.head()


,dmca_id,dmca_type,dmca_title,dmca_body,dmca_date_sent,dmca_date_received,dmca_topics_0,dmca_topics_1,dmca_sender_name,dmca_principal_name,dmca_recipient_name,dmca_works_0_description,dmca_tags,dmca_jurisdictions_0,dmca_action_taken,dmca_language,dmca_case_id_number,dmca_works_1_description,dmca_works_2_description,dmca_works_3_description,dmca_works_4_description,dmca_works_5_description,dmca_works_6_description,dmca_works_7_description,dmca_works_8_description,dmca_works_9_description,infringing_url,copyrighted_url
0,30194140,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-04T00:00:00.000Z,2023-01-04T00:00:00.000Z,DMCA Notices,Copyright,Pengetahuan-ID,None,Google LLC,Artikel blog saya dicuri yang berjudul Kumpula...,[],ID,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hukum96.com/2021/08/wallpaper-cosp...,https://pengetahuandi.blogspot.com/2019/12/kum...
1,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2021/05/trieu...
2,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2020/05/dung-...
3,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2021/12/cham-...
4,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2021/05/cach-...


In [143]:
tidy_data.shape

(3408502, 28)

In [144]:
# Keeping only rows that have a blog as their copyrighted_url
filtered_data = tidy_data[
    tidy_data['copyrighted_url'].str.contains('|'.join(blog_words), na=False)
]

In [145]:
filtered_data.head()

,dmca_id,dmca_type,dmca_title,dmca_body,dmca_date_sent,dmca_date_received,dmca_topics_0,dmca_topics_1,dmca_sender_name,dmca_principal_name,dmca_recipient_name,dmca_works_0_description,dmca_tags,dmca_jurisdictions_0,dmca_action_taken,dmca_language,dmca_case_id_number,dmca_works_1_description,dmca_works_2_description,dmca_works_3_description,dmca_works_4_description,dmca_works_5_description,dmca_works_6_description,dmca_works_7_description,dmca_works_8_description,dmca_works_9_description,infringing_url,copyrighted_url
0,30194140,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-04T00:00:00.000Z,2023-01-04T00:00:00.000Z,DMCA Notices,Copyright,Pengetahuan-ID,None,Google LLC,Artikel blog saya dicuri yang berjudul Kumpula...,[],ID,None,None,None,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,https://www.hukum96.com/2021/08/wallpaper-cosp...,https://pengetahuandi.blogspot.com/2019/12/kum...
1,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2021/05/trieu...
2,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2020/05/dung-...
3,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2021/12/cham-...
4,30150668,DMCA,DMCA (Copyright) Complaint to Google,None,2023-01-01T00:00:00.000Z,2023-01-01T00:00:00.000Z,DMCA Notices,Copyright,Vũ Đoàn,None,Google LLC,The thief changed the URLs of the previously r...,[],VN,None,None,None,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,The thief changed the URLs of the previously r...,Copyright of the entire article content,Copyright of the entire article content,https://yeumaivang.com/benh-ri-sat-tren-cay-mai/,https://hoamaixunau.blogspot.com/2021/05/cach-...


In [146]:
clean_data = filtered_data [[
    'dmca_id',
    'dmca_date_sent',
    'dmca_sender_name',
    'dmca_works_0_description',
    'dmca_jurisdictions_0',
    'copyrighted_url',
    'infringing_url'
]]

In [147]:
clean_data.sample(5)

,dmca_id,dmca_date_sent,dmca_sender_name,dmca_works_0_description,dmca_jurisdictions_0,copyrighted_url,infringing_url
216022,30343193,2023-01-17T00:00:00.000Z,الوظائف والأون لاين,هذا المنتدى قام بسرقة عدد كبير من مقالاتي ونسخ...,EG,https://wazaef-online.blogspot.com/2020/03/Man...,https://forum.janatna.com/t20028
3060967,34144252,2023-05-23T00:00:00.000Z,DMCA START ROUND 1,This website steals images from our website. a...,IN,https://www.google.com/search?num=100&sxsrf=AP...,https://callsexy.in/
227112,32636993,2023-02-20T00:00:00.000Z,الوظائف والأون لاين,تقوم هذه المواقع بسرقة محتوى مقالاتي التي أضع ...,EG,https://wazaef-online.blogspot.com/2020/10/suc...,https://www.zainalbahes.com/2022/09/freelancer...
223560,30494362,2023-01-29T00:00:00.000Z,الوظائف والأون لاين,هذا المنتدى قام بسرقة عدد كبير من مقالاتي ونسخ...,EG,https://wazaef-online.blogspot.com/2020/01/ama...,https://forum.janatna.com/t19998-%D8%AA%D8%B9%...
224598,30494362,2023-01-29T00:00:00.000Z,الوظائف والأون لاين,هذا المنتدى قام بسرقة عدد كبير من مقالاتي ونسخ...,EG,https://wazaef-online.blogspot.com/2020/04/fre...,https://forum.janatna.com/t19985-%D9%83%D9%8A%...


26,615 results. We are getting there, but there's still some work to be done. Let's remove all rows that have a blog as their `infringing_url`

In [106]:
clean_data.tail()

,dmca_id,dmca_date_sent,dmca_sender_name,dmca_works_0_description,dmca_jurisdictions_0,copyrighted_url,infringing_url
3408497,35184501,2023-07-27T00:00:00.000Z,Danko River,Website chungcuhn24h.net đã copy nội dung bàn ...,VN,https://dankoriver.blogspot.com/2023/07/vi-tri...,https://chungcuhn24h.net/danko-river-phu-binh/
3408498,35152680,2023-07-25T00:00:00.000Z,Mohammad Al Fatih,l am the real owner of this Image. My image is...,BD,https://mohammad-al-fatih.blogspot.com/2016/07...,https://mlwbd.page/movie/anegan-2015-480p-720p...
3408499,35122843,2023-07-23T00:00:00.000Z,[REDACTED],[REDACTED],GR,https://www.tumblr.com/reportdaily5/7218074523...,https://cookcountyrecord.com/stories/511633190...
3408500,35173636,2023-07-26T00:00:00.000Z,Беларуская Брама,[REDACTED],PL,https://breadrusax.wixsite.com/belaruskaya-brama,https://yegor.su/music/%D0%B1%D0%B5%D0%BB%D0%B...
3408501,35123385,2023-07-23T00:00:00.000Z,Villete Brunelle,"""El juez de la Audiencia Nacional de España Jo...",CA,https://www.tumblr.com/genztraders/72370469110...,https://elcryptografo.uy/espana-ordenan-captur...


In [148]:
# Removing all rows that have a blog as their infrining_url (=likely not legit journalism)
clean_data = clean_data[
    ~clean_data['infringing_url'].str.contains('|'.join(blog_words), na=False)
]


In [149]:
clean_data.shape

(25253, 7)

Let's see what platforms were targeted and see if we can find something.

# 2. Adding `infringing_source` and `copyrighted_source`

In [150]:
import re
import numpy

## 2.1. Infringing_source

In [152]:
# Define the regular expression pattern to match the source URL
pattern = r'(https?://[^/]+)'

# Use the pattern to extract the source URL and create a new column
clean_data['source_infringing'] = clean_data['infringing_url'].str.extract(pattern)


In [153]:
clean_data.sample(3)

,dmca_id,dmca_date_sent,dmca_sender_name,dmca_works_0_description,dmca_jurisdictions_0,copyrighted_url,infringing_url,source_infringing
3024771,34020085,2023-05-17T00:00:00.000Z,Beryl Kerning Consulting Corporation,"Dear Sir / Madam, my name is Momchil Petrov an...",US,https://www.besteonlinecasinos.com/wixstars/,http://nachttopf.de/casino-com/,http://nachttopf.de
613574,32755668,2023-02-28T00:00:00.000Z,"ООО ""Веб Фокус""","Сайт компании ООО ""Веб Фокус""",BY,https://web-f.ru/razrabotka-sajta-na-wix/,https://web-studio-piter.ru/prodvizhenie-insta...,https://web-studio-piter.ru
223053,30494362,2023-01-29T00:00:00.000Z,الوظائف والأون لاين,هذا المنتدى قام بسرقة عدد كبير من مقالاتي ونسخ...,EG,https://wazaef-online.blogspot.com/2022/07/how...,https://forum.janatna.com/t20008-%D9%83%D9%8A%...,https://forum.janatna.com


In [159]:
# Grouping by the 'source_infringing' column and counting the frequency
infringing_url_count = clean_data.groupby('source_infringing').size().reset_index(name='count')

# Sorting the result by frequency in descending order to see the most frequent source URLs first
infringing_url_count = infringing_url_count.sort_values(by='count', ascending=False)

# Displaying the result
infringing_url_count.tail(20)

,source_infringing,count
392,https://smartguides24.com,1
391,https://smartaffiliatesuccess.com,1
390,https://siteskill.com.ng,1
388,https://simanto.jisan-404.com,1
386,https://shayariurdu.com,1
385,https://sexhd.pics,1
383,https://secretocapital.com,1
355,https://qz.com,1
382,https://scooprdc.net,1
377,https://saudileaks.org,1


## 2.2 Copyrighted_source

In [154]:
# Define the list of blog words
blog_words = [
    'blogspot',
    'tumblr',
    'livejournal',
    'issuu',
    'wix',
    'over-blog'
]

# Define the conditions and corresponding sources
conditions = [clean_data['copyrighted_url'].str.contains(word, na=False) for word in blog_words]

# Use np.select to apply the conditions and create the 'source_copyrighted' column
clean_data['source_copyrighted'] = np.select(conditions, blog_words, default='other')

clean_data.sample(3)


,dmca_id,dmca_date_sent,dmca_sender_name,dmca_works_0_description,dmca_jurisdictions_0,copyrighted_url,infringing_url,source_infringing,source_copyrighted
228300,32636993,2023-02-20T00:00:00.000Z,الوظائف والأون لاين,تقوم هذه المواقع بسرقة محتوى مقالاتي التي أضع ...,EG,https://wazaef-online.blogspot.com/2020/10/ads...,https://forum.janatna.com/t20014-%D9%83%D9%8A%...,https://forum.janatna.com,blogspot
228308,32636993,2023-02-20T00:00:00.000Z,الوظائف والأون لاين,تقوم هذه المواقع بسرقة محتوى مقالاتي التي أضع ...,EG,https://wazaef-online.blogspot.com/2022/02/sma...,https://forum.janatna.com/t20014-%D9%83%D9%8A%...,https://forum.janatna.com,blogspot
227259,32636993,2023-02-20T00:00:00.000Z,الوظائف والأون لاين,تقوم هذه المواقع بسرقة محتوى مقالاتي التي أضع ...,EG,https://wazaef-online.blogspot.com/2020/02/Com...,https://arwordshub.com/%D9%83%D9%8A%D9%81-%D8%...,https://arwordshub.com,blogspot


In [161]:
# Grouping by the 'source_infringing' column and counting the frequency
source_copyrighted_count = clean_data.groupby('source_copyrighted').size().reset_index(name='count')

# Sorting the result by frequency in descending order to see the most frequent source URLs first
source_copyrighted_count = source_copyrighted_count.sort_values(by='count', ascending=False)

# Displaying the result
source_copyrighted_count

,source_copyrighted,count
0,blogspot,17848
5,wix,5505
4,tumblr,1870
3,over-blog,13
1,issuu,10
2,livejournal,7


## 3. Removing most common `infringing_source`

In [164]:
infringing_url_count.head(10)

,source_infringing,count
213,https://forum.janatna.com,9730
591,https://www.hukum96.com,2275
21,http://wandamalfara.com,1996
226,https://graceprivateforwomen.com,998
58,https://99fx.net,871
89,https://arwordshub.com,807
96,https://beelook.net,700
399,https://spinszilla.org,581
126,https://captions.tellygupshup.com,462
463,https://web-studio-piter.ru,434


Most takedown requests targeted a forum and some sketchy websites which I am not interested in. Because I am looking for pieces of journalism that were removed, I am probably not interested in all those blogs and sketchy websites. Let's remove them. Let's say I want to remove from my dataset all `infringing_source` (websites) that appears more than ten times.

In [167]:
# Filtering the source_url_frequency DataFrame to keep only the rows with a frequency higher than 50
infringing_keep = source_infringing_count[source_infringing_count['count'] < 10]

# Displaying the result
infringing_keep


,source_infringing,count
754,https://yt3.ggpht.com,9
732,https://www.womenchannel.org,8
632,https://www.masterfap.net,8
509,https://www.camwhores.video,8
378,https://scamrecovery.net,8
...,...,...
366,https://reportscam.net,1
365,https://replit.com,1
357,https://radiowest.kuer.org,1
356,https://radar.avrotros.nl,1


In [168]:
urls_to_keep = infringing_keep['source_infringing']
urls_to_keep

754           https://yt3.ggpht.com
732    https://www.womenchannel.org
632       https://www.masterfap.net
509     https://www.camwhores.video
378        https://scamrecovery.net
                   ...             
366          https://reportscam.net
365              https://replit.com
357      https://radiowest.kuer.org
356       https://radar.avrotros.nl
757              https://zthots.com
Name: source_infringing, Length: 630, dtype: object

In [169]:
# Filter the tidy_data DataFrame to keep only the rows with a source_url in the above list
clean_data1 = clean_data[clean_data['source_infringing'].isin(urls_to_keep)]

clean_data1


,dmca_id,dmca_date_sent,dmca_sender_name,dmca_works_0_description,dmca_jurisdictions_0,copyrighted_url,infringing_url,source_infringing,source_copyrighted
234,30176050,2023-01-03T00:00:00.000Z,Raining Romance LLC,Its my book. A black background with gold crac...,US,https://ignite-books3.blogspot.com/2022/11/ser...,https://vk.com/wall329000261_1450,https://vk.com,blogspot
239,30176050,2023-01-03T00:00:00.000Z,Raining Romance LLC,Its my book. A black background with gold crac...,US,https://ignite-books3.blogspot.com/2022/11/ser...,https://www.epub.pub/book/eternally-hers-by-ja...,https://www.epub.pub,blogspot
244,30176050,2023-01-03T00:00:00.000Z,Raining Romance LLC,Its my book. A black background with gold crac...,US,https://ignite-books3.blogspot.com/2022/11/ser...,https://forum.mobilism.me/viewtopic.php?f=1292...,https://forum.mobilism.me,blogspot
254,30176050,2023-01-03T00:00:00.000Z,Raining Romance LLC,Its my book. A black background with gold crac...,US,https://ignite-books3.blogspot.com/2022/11/ser...,https://forum.mobilism.me/viewtopic.php?f=1292...,https://forum.mobilism.me,blogspot
256,32934442,2023-01-05T00:00:00.000Z,[REDACTED],[REDACTED],ES,https://www.tumblr.com/24newspressblog/7055967...,https://es.cointelegraph.com/news/top-criptono...,https://es.cointelegraph.com,tumblr
...,...,...,...,...,...,...,...,...,...
3408497,35184501,2023-07-27T00:00:00.000Z,Danko River,Website chungcuhn24h.net đã copy nội dung bàn ...,VN,https://dankoriver.blogspot.com/2023/07/vi-tri...,https://chungcuhn24h.net/danko-river-phu-binh/,https://chungcuhn24h.net,blogspot
3408498,35152680,2023-07-25T00:00:00.000Z,Mohammad Al Fatih,l am the real owner of this Image. My image is...,BD,https://mohammad-al-fatih.blogspot.com/2016/07...,https://mlwbd.page/movie/anegan-2015-480p-720p...,https://mlwbd.page,blogspot
3408499,35122843,2023-07-23T00:00:00.000Z,[REDACTED],[REDACTED],GR,https://www.tumblr.com/reportdaily5/7218074523...,https://cookcountyrecord.com/stories/511633190...,https://cookcountyrecord.com,tumblr
3408500,35173636,2023-07-26T00:00:00.000Z,Беларуская Брама,[REDACTED],PL,https://breadrusax.wixsite.com/belaruskaya-brama,https://yegor.su/music/%D0%B1%D0%B5%D0%BB%D0%B...,https://yegor.su,wix


Now we have a condensed dataset of 1,631 rows. To see if there are takedowns that are interesting for our research, we are going to evaluate them one-by-one in Google Sheets.

In [170]:
clean_data1.to_csv('data/clean_data1.csv')

The Google Sheet can is available at the following link: https://docs.google.com/spreadsheets/d/1PkJy7dFvoSeltSoXpe2EcXKVoScQheMzlwKtXImdDU0/edit?usp=sharing